In [1]:
import evaluate

In [32]:
accuracy = evaluate.load("accuracy")
f1_micro = evaluate.load("f1")
f1_macro = evaluate.load("f1")

In [33]:
y_pred = [[0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1]]
y_true = [[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]]

In [34]:
# transform from one hot encoding to class number using argmax
y_pred = [max(enumerate(i), key=lambda x: x[1])[0] for i in y_pred]
y_true = [max(enumerate(i), key=lambda x: x[1])[0] for i in y_true]

In [35]:
y_pred, y_true

([3, 0, 4], [2, 2, 4])

In [36]:
accuracy.add_batch(predictions=y_pred, references=y_true)
f1_micro.add_batch(predictions=y_pred, references=y_true)
f1_macro.add_batch(predictions=y_pred, references=y_true)

In [37]:
accuracy.compute(), f1_macro.compute(average="macro"), f1_micro.compute(average="micro")

({'accuracy': 0.3333333333333333}, {'f1': 0.25}, {'f1': 0.3333333333333333})